In [4]:
import os
from dotenv import load_dotenv

# To use OpenAI, do the following:
# Create a .env file and add your api key:
##   OPENAI_API_KEY=<your_api_key>
load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models.ollama import ChatOllama 

# llm = ChatOpenAI()
llm = ChatOllama(model="llama2", base_url=os.environ["OLLAMA_BASE_URL"])

In [6]:
query = "how can langsmith help with testing?"
llm.invoke(query)

AIMessage(content="\nLangsmith is a language model that can assist with testing in several ways:\n\n1. Automated Testing: Langsmith can be used to automate testing processes, such as unit testing and integration testing. It can generate test cases based on the requirements of the software being developed, and run them automatically to identify any defects or issues.\n2. Test Data Generation: Langsmith can be used to generate test data that can be used to test the software. For example, it can generate user inputs, Edge cases, and boundary values that can be used to test the software's functionality.\n3. Test Case Design: Langsmith can assist in designing test cases by suggesting different approaches and strategies for testing. It can also help in identifying the most critical test cases that cover a large part of the software's functionality.\n4. Defect Prediction: Langsmith can be used to predict potential defects in the software based on its analysis of the code and the requirements.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

In [8]:
chain = prompt | llm

In [9]:
message = chain.invoke({"input": query})
message.content

'As a world-class technical documentation writer, I can assist in various ways during the testing process:\n\n1. Content Creation: Langsmith can create clear, concise, and comprehensive test plans, test cases, and test scripts to ensure that your software or system is thoroughly tested. These documents will help you identify and fix any issues before the product launches.\n2. Testing Support: As a writer, I can provide testing support by creating documentation for testing tools, such as automated testing frameworks or load testing tools. This documentation can include user manuals, quick-start guides, and API documentation to help your team use these tools effectively.\n3. Quality Assurance (QA) Documentation: Langsmith can create QA documentation, such as test plans, test cases, and bug reports, to ensure that the software or system meets the required quality standards. This documentation will help you identify and fix any issues before the product is released.\n4. User Acceptance Tes

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [11]:
chain = prompt | llm | output_parser

In [12]:
response = chain.invoke({"input": query})
print(response)

As a world-class technical documentation writer, I can provide valuable insights and tools to help testers in the following ways:

1. Creating Test Plans: With my expertise in writing, I can assist testers in creating comprehensive test plans that cover all aspects of a product or feature. This includes identifying test cases, prioritizing them based on risk and complexity, and outlining the resources required for testing.
2. Developing Test Cases: I can help testers develop clear, concise, and comprehensive test cases that cover various scenarios and use cases. This ensures that all aspects of a product or feature are thoroughly tested, and any defects or issues are identified early on.
3. Creating Test Scripts: As a technical writer, I can create detailed test scripts that outline the steps required to perform a particular test case. This includes identifying the inputs, expected outputs, and any Edge Cases that need to be addressed.
4. Providing Technical Guidance: My expertise in t

In [13]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
docs

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for 

In [14]:
from langchain_openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [15]:
from langchain_community.vectorstores.chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)
vector_db = Chroma.from_documents(documents, embed_model)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """Answer the question based only on the given context:

<context>
{context}
</context>

Question: {input}"""
prompt = ChatPromptTemplate.from_template(template)

document_chain = create_stuff_documents_chain(llm, prompt)

In [17]:
from langchain_core.documents import Document

document_chain.invoke({
    "context": [Document(page_content="Langsmith can help you visualize test results and track metrics.")],
    "input": query 
})

'\nBased on the given context, Langsmith can help with testing by providing visualization and tracking of test results.'

In [18]:
from langchain.chains.retrieval import create_retrieval_chain

retriever = vector_db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrieval_chain.invoke({"input": query})
print(response["answer"])

Based on the given context, LangSmith can help with testing in the following ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, that can be used to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
2. Running custom evaluations: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results. This allows developers to evaluate their LLM applications with different evaluation metrics, such as latency, cost, and feedback scores.
3. Debugging: When developing new LLM applications, LangSmith tracing is enabled by default. This provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues. Additionally, LangSmith allows users to grow benchmarking datasets, annotate traces, and drill down into important data in tra